# Preprocessing data channels

In [40]:
import json
import pandas as pd
import numpy as np
import pyedflib
from pathlib import Path

### Pandas options

In [41]:
pd.set_option('max_row', None)
pd.set_option('max_columns', None)

## Channels and expereiments info

cha_dict = {'F3..' : '32', 'F6..': '37', 'P3..': '49', 'P6..' :'54'}

exp_dict = {'03' : ['D_E_',  'R' ], '04' : ['D_E_',  'I' ] , '05' : ['C_B_',  'R' ], '06' : ['C_B_',  'I' ]}

## File paths

In [42]:
root = Path()
info_path = root / 'info'
data_path = root /'data'
raw_data_path = data_path / 'data_down'
preproc_path = data_path / 'data_pre_proc'
preproc_path.mkdir(exist_ok=True)

### Experiment info dict (loading)

In [43]:
exp_dict_file = info_path / 'experiment_dict.json'
with open(exp_dict_file) as f:
   exp_dict = json.load(f)
f.close()
exp_dict

{'01': {'Experiment': 1, 'Type': ' Baseline', 'Activity': 'Eyes Open'},
 '02': {'Experiment': 2, 'Type': ' Baseline', 'Activity': 'Eyes Closed'},
 '03': {'Experiment': 3, 'Type': ' Real', 'Activity': 'Left Right'},
 '04': {'Experiment': 4, 'Type': ' Imaginary', 'Activity': 'Left Right'},
 '05': {'Experiment': 5, 'Type': ' Real', 'Activity': 'Top Down'},
 '06': {'Experiment': 6, 'Type': ' Imaginary', 'Activity': 'Top Down'},
 '07': {'Experiment': 7, 'Type': ' Real', 'Activity': 'Left Right'},
 '08': {'Experiment': 8, 'Type': ' Imaginary', 'Activity': 'Left Right'},
 '09': {'Experiment': 9, 'Type': ' Real', 'Activity': 'Top Down'},
 '10': {'Experiment': 10, 'Type': ' Imaginary', 'Activity': 'Top Down'},
 '11': {'Experiment': 11, 'Type': ' Real', 'Activity': 'Left Right'},
 '12': {'Experiment': 12, 'Type': ' Imaginary', 'Activity': 'Left Right'},
 '13': {'Experiment': 13, 'Type': ' Real', 'Activity': 'Top Down'},
 '14': {'Experiment': 14, 'Type': ' Imaginary', 'Activity': 'Top Down'}}

In [44]:
exp_num = '05'
' : '.join([exp_dict[exp_num]['Type'], exp_dict[exp_num]['Activity']])

' Real : Top Down'

In [45]:
cha_dict_file = info_path / 'channels_code.json'
with open(cha_dict_file) as f:
   cha_dict_aux = json.load(f)
f.close()
cha_dict = {}
for i in cha_dict_aux:
   cha_dict[i['Code']] = i['Number']
cha_dict

{'F3..': 32, 'F6..': 37, 'P3..': 49, 'P6..': 54}

## Extracting channels data

In [48]:
edf_folders = raw_data_path.glob('*')

cut_len = 19_000

for folder in edf_folders:
    sub_proc_path = preproc_path / folder.stem
    sub_proc_path.mkdir(exist_ok=True)
    edf_files = folder.glob("*.edf")
    for file_path in edf_files:
        exp_num = file_path.stem.split('R')[-1]
        exp_path = sub_proc_path / '_'.join([folder.stem, exp_num])
        exp_path.mkdir(exist_ok=True)
        # exporting selected channels
        f = pyedflib.EdfReader(str(file_path))
        n = f.signals_in_file
        signal_labels = f.getSignalLabels()
        #print(n, f.getNSamples() , signal_labels)
        # extracting   channels
        sigbufs = np.zeros((n, f.getNSamples()[0]))
        for i in np.arange(n):
                #print(i)
                sigbufs[i, :] = f.readSignal(i)
        # close file
        f.close()
        
        print(' Subject: {}, Array shape: {}, Experiment number: {}, Experiment info: {}'.format(  folder.stem, sigbufs.shape, exp_num, ' - '.join([ exp_dict[exp_num]['Activity'], exp_dict[exp_num]['Type']]) ))
        df = pd.DataFrame(sigbufs.T, columns= signal_labels)
        df = df.head(cut_len) # cortando as series
        for i in cha_dict.keys():
            df_aux = df[i]
            file_name = exp_path / ("_".join([folder.stem , exp_num , str(cha_dict[i])]) + '.txt')
            df_aux.to_csv(file_name, header=False, index=False)

 Subject:S001, Array shape:(64, 20000), Experiment number:03, Experiment info:Left Right -  Real
 Subject:S001, Array shape:(64, 20000), Experiment number:04, Experiment info:Left Right -  Imaginary
 Subject:S001, Array shape:(64, 20000), Experiment number:05, Experiment info:Top Down -  Real
 Subject:S001, Array shape:(64, 20000), Experiment number:06, Experiment info:Top Down -  Imaginary
 Subject:S001, Array shape:(64, 20000), Experiment number:07, Experiment info:Left Right -  Real
 Subject:S001, Array shape:(64, 20000), Experiment number:08, Experiment info:Left Right -  Imaginary
 Subject:S001, Array shape:(64, 20000), Experiment number:09, Experiment info:Top Down -  Real
 Subject:S001, Array shape:(64, 20000), Experiment number:10, Experiment info:Top Down -  Imaginary
 Subject:S001, Array shape:(64, 20000), Experiment number:11, Experiment info:Left Right -  Real
 Subject:S001, Array shape:(64, 20000), Experiment number:12, Experiment info:Left Right -  Imaginary
 Subject:S001